# Cohort Forecaster

Forecast volume by inputting user cohorts & a pre-defined contact distribtion.

In [1]:
import Bens_forecasting_utils as fc
import pandas as pd
import numpy as np
import logging
logging.basicConfig(format='%(levelname)s: %(message)s') 
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pandas.tseries.offsets import MonthEnd
%matplotlib ipympl
plt.rcParams['figure.figsize'] = [10, 5] # make default plot size bigger

RESERVED_COL_NAMES = pd.Series(['date', 'forecast'])

### Set Parameters here:

In [2]:
parameters = dict(
    
    # cs kyc forecast converter:
    sheet_id                    = '1a_v0g2y5DvqLDbXXJC0m7i3PjF2AseC3lqoRwlGxg1o',
    raw_marketing_forecast      = 'raw_marketing_forecast!A1:R',
    tnc_to_lang                 = 'Tnc_to_Lang!A1:F',
    intra_month                 = 'kycc_by_lang_intra-month_seasonality!A1:D',
    intra_week                  = 'kycc_by_lang_intra-week_seasonality!A1:E' ,
    cohorts                     = 'cohorts!A1:B',
    kycc_volume                 = 'kycc_volume!A1:F'
)

In [3]:
# Name of the column to match the cohorts to the vol distribution (must exist in both sheets)
grouping_column           = 'business_line_alias'

# If left blank will by default output to the cohorts input sheet in a tab called 'output_fc'
output_sheet_id           = ''
output_sheet_range        = ''

# If set to true then the forecast will still proceed if some cohorts don't have a matching vol distro.
# If not true then an error will be thrown if cohorts with no matching vol distro are found.
allow_missing_vol_distros = True

**Note:** first time run will prompt authorisation with Google SSO for the G-Sheets API. After that a `token.json` file will be stored locally containing API credentials.

Credentials will remain valid unless they are not used for 6 months (or if the API scope is changed in `Bens_forecasting_utils.py`). In this case you may need to manually delete `token.json` to trigger the Google SSO authorisation flow upon the next execution.

---

### Forecast Script Start:
###### Import G-Sheets with cohorts data & vol distribution, check for errors:

### generating df_cohort

#### converting marketing forecast kycc to daily kycc cohorts

In [4]:
def generate_dates(month_year):

    
    date = pd.to_datetime(month_year, format='%b %y', errors='coerce')
    
    first_day = date
    last_day = date + MonthEnd(1)
    
    return pd.date_range(start=first_day, end=last_day).to_list()

In [5]:
def cohort_df(sheet_id=parameters['sheet_id'],
              raw_marketing_forecast=parameters['raw_marketing_forecast'],
              tnc_to_lang=parameters['tnc_to_lang'],
              intra_month=parameters['intra_month'],
              intra_week=parameters['intra_week'],
              cohorts=parameters['cohorts']):
    
    # importing raw_marketing forecast and converting it to daily numbers
    df = fc.import_gsheet_to_df(sheet_id, raw_marketing_forecast) #importing data from gsheet
    df.iloc[:,1:] = df.iloc[:,1:].replace(',','',regex=True).astype('int32') #converting integers
    
    df = df[df['KYCC']!='Total'] #removing total
    
    c_code = {'Germany':'DEU','Austria':'AUT','Spain':'ESP','France':'FRA','Italy':'ITA','Spain':'ESP','Greater Europe':'GrE',
          'Non-Euro':'NEuro'}
    
    df['TnC Country'] = df['KYCC'].map(c_code) #creating KYCC columns
    
    df = df.melt(id_vars=['KYCC','TnC Country'],var_name='Month',value_name='Total')
    
    # import tnc 
    tnc = fc.import_gsheet_to_df(sheet_id, tnc_to_lang)
    
    tnc.iloc[:,1:] = tnc.iloc[:,1:].replace('%','',regex=True).astype('float')
    
    merged_df = pd.merge(df, tnc, on='TnC Country')
    
    for col in tnc.columns:
        if col != 'TnC Country':
            merged_df[col] = merged_df['Total'] * (merged_df[col] / 100)
            
    df_melt = pd.melt(merged_df, id_vars=['Month'], value_vars=['de','en','es','fr','it'], var_name='language',
        value_name='value')
    
    df_melt = df_melt.groupby(['Month','language'])['value'].sum().reset_index()
    
    df_melt['date'] = df_melt['Month'].str.strip().apply(generate_dates)
    
    data = df_melt.explode('date').reset_index(drop=True)
    
    data = data.sort_values(by=['date','language'])
    
    data['dom'] = data['date'].dt.day
    data['dow'] = data['date'].dt.weekday + 1
    data.rename(columns={'language':'business_line_alias'},inplace=True)
    
    intra_m = fc.import_gsheet_to_df(sheet_id, intra_month)
    intra_w = fc.import_gsheet_to_df(sheet_id, intra_week)
    
    intra_m['dom'] = intra_m['dom'].astype('int32')
    intra_w['dow'] = intra_w['dow'].astype('int32')
    
    data = pd.merge(data, intra_m, how='left', on=['dom','business_line_alias'])
    data = pd.merge(data, intra_w, how='left', on=['dow','business_line_alias'])
    data.rename(columns={'seasonality':'intra_week_seasonality'},inplace=True)
    data = data[['Month','date','business_line_alias','value','dom','dow','intra_month_seasonality','intra_week_seasonality']]
    data['daily'] = data['value'] / data['date'].dt.to_period('M').dt.days_in_month
    
    
    data[['daily','intra_month_seasonality','intra_week_seasonality']] = data[['daily','intra_month_seasonality','intra_week_seasonality']].astype('float')
    data['daily_adj'] = (data['daily'] * (1+data['intra_month_seasonality'])) * (1+data['intra_week_seasonality'])
    data['daily_adj_total'] = data.groupby(['Month','business_line_alias'])['daily_adj'].transform('sum')
    data['final_daily'] = (1+(data['value'] - data['daily_adj_total']) / data['daily_adj_total']) * data['daily_adj']
    data = data[['date','business_line_alias','final_daily']]
    data.rename(columns={'business_line_alias':'language'},inplace=True)
    
    # getting cohorts
    cohorts_1 = fc.import_gsheet_to_df(sheet_id, cohorts)
    df_cohorts = pd.merge(data, cohorts_1, how='left', on='language')
    df_cohorts.rename(columns={'date':'cohort_start_date','final_daily':'cohort_size'},inplace=True)
    df_cohorts = df_cohorts[['cohort_start_date','business_line_alias','cohort_size']]
    
    return df_cohorts

In [6]:
df_cohorts = cohort_df()

In [18]:
df_cohorts

,cohort_start_date,business_line_alias,cohort_size
0,2024-08-01,ops-cs-L1-de-call,821.0854
1,2024-08-01,ops-cs-L1-de-chat,821.0854
2,2024-08-01,ops-cs-L1-de-email,821.0854
3,2024-08-01,ops-cs-L1-en-call,913.988116
4,2024-08-01,ops-cs-L1-en-chat,913.988116
...,...,...,...
7765,2025-12-31,ops-cs-L1-fr-chat,1232.055069
7766,2025-12-31,ops-cs-L1-fr-email,1232.055069
7767,2025-12-31,ops-cs-L1-it-call,446.64227
7768,2025-12-31,ops-cs-L1-it-chat,446.64227


### generating df_vol_distro, kycc contact rates by business_line_alias


In [7]:
def vol_distro(sheet_id=parameters['sheet_id'],
              kycc_volume=parameters['kycc_volume']):
    
    # importing kycc_
    kycc = fc.import_gsheet_to_df(sheet_id, kycc_volume)
    kycc['business_line_alias'] = 'ops-cs-L1-' + kycc['contact_language'] + '-' + kycc['channel']
    
    df_vol_distro = kycc[['days_since_kycc','business_line_alias','cs_contact_rate']]
    
    df_vol_distro = df_vol_distro.rename(columns={'days_since_kycc':'date_offset','cs_contact_rate':'vol_distro'})
    
    return df_vol_distro

In [8]:
df_vol_distro = vol_distro()

In [19]:
df_vol_distro

,date_offset,business_line_alias,vol_distro
1,-4,ops-cs-L1-de-call,0.00000000000
2,-3,ops-cs-L1-de-call,0.00000000000
3,-2,ops-cs-L1-de-call,0.00000000000
4,-1,ops-cs-L1-de-call,0.00000000000
5,0,ops-cs-L1-de-call,0.00104341821
...,...,...,...
596,31,ops-cs-L1-it-email,0.00344561584
597,32,ops-cs-L1-it-email,0.00298277192
598,33,ops-cs-L1-it-email,0.00195422988
599,34,ops-cs-L1-it-email,0.00298277192


In [9]:
#df_cohorts = fc.import_gsheet_to_df(cohorts_sheet_id, cohorts_sheet_range)
#df_vol_distro = fc.import_gsheet_to_df(vol_distro_sheet_id, vol_distro_sheet_range)

#check imported data for reserved column names
illegal_cols = pd.Series(list(set(RESERVED_COL_NAMES) & set(df_cohorts.keys())), dtype='str')
illegal_cols = pd.Series(pd.concat([illegal_cols, pd.Series(list(set(RESERVED_COL_NAMES) & set(df_vol_distro.keys())), dtype='str')]).unique())
if len(illegal_cols) > 0:
    illegal_cols_concat = '\'' + illegal_cols.str.cat(sep='\', \'') + '\''
    reserved_cols_concat = '\'' + RESERVED_COL_NAMES.str.cat(sep='\', \'') + '\''
    logging.error('Found column: ' + illegal_cols_concat + ' in the source sheet. These column names are reserved: ' + reserved_cols_concat + ' for use by the script. Please rename! ❤️')
    raise Exception('Reserved column names found in import data.')

#check if there are any business lines in the cohorts without a corresponding volume distribution
missing_distros = fc.check_orphan_groups(df_cohorts, df_vol_distro, grouping_column, behaviour='')
if missing_distros:
    missing_groups_concat = '[' + missing_distros.str.cat(sep='\', \'') + ']'
    if allow_missing_vol_distros:
        logging.warning('Missing volume distribution for following business line(s): ' + missing_groups_concat + '\n🚨 Cohorts from these business lines will be missing from the output!!')
    else:
        logging.error('Missing volume distribution for following business line(s): ' + missing_groups_concat)
        raise Exception('Missing volume distribution for following business line(s): ' + missing_groups_concat)

print('# of cohorts: ' + str(df_cohorts['cohort_start_date'].count()))
print('# of business lines: ' + str(df_cohorts[grouping_column].nunique()))

# of cohorts: 7770
# of business lines: 15


---
###### Join each cohort with corresponding volume distribution to create forecast:

In [10]:
%%time
# merge cohorts with volume distribution & calculate forecast values
df_fc = df_cohorts.merge(df_vol_distro, left_on=[grouping_column], right_on=[grouping_column], how='inner')
df_fc['cohort_start_date'] = pd.to_datetime(df_fc['cohort_start_date'], format='%d/%m/%Y')
df_fc['cohort_size'] = pd.to_numeric(df_fc['cohort_size'])
df_fc['vol_distro'] = pd.to_numeric(df_fc['vol_distro'])
df_fc['date_offset'] = pd.to_numeric(df_fc['date_offset'])
print(max(df_fc['cohort_start_date']))

df_fc['forecast_date'] = df_fc['cohort_start_date'] + pd.to_timedelta(df_fc['date_offset'], unit='D') #causes perf warning but idk a more efficient way to do this 🤷‍♂️
df_fc['forecast_base'] = df_fc['cohort_size'] * df_fc['vol_distro']

print(max(df_fc['forecast_date']))

df_fc_grouped = df_fc.groupby(['forecast_date', 'business_line_alias']).agg(
    forecast = pd.NamedAgg(column='forecast_base', aggfunc=sum)
    #cohort_count = pd.NamedAgg(column='cohort_size', aggfunc=len)
)
print(df_fc_grouped)

2025-12-31 00:00:00
2026-02-04 00:00:00
                                   forecast
forecast_date business_line_alias          
2024-07-28    ops-cs-L1-de-call    0.000000
              ops-cs-L1-de-chat    0.000000
              ops-cs-L1-de-email   0.000000
              ops-cs-L1-en-call    0.000000
              ops-cs-L1-en-chat    0.000000
...                                     ...
2026-02-04    ops-cs-L1-fr-chat    1.294291
              ops-cs-L1-fr-email   2.325336
              ops-cs-L1-it-call    0.114848
              ops-cs-L1-it-chat    0.895811
              ops-cs-L1-it-email   0.597207

[8355 rows x 1 columns]
CPU times: user 444 ms, sys: 17.4 ms, total: 462 ms
Wall time: 461 ms


<timed exec>:14: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.


In [20]:
df_fc_grouped

,forecast_date,business_line_alias,forecast
0,2024/07/28,ops-cs-L1-de-call,0.000000
1,2024/07/28,ops-cs-L1-de-chat,0.000000
2,2024/07/28,ops-cs-L1-de-email,0.000000
3,2024/07/28,ops-cs-L1-en-call,0.000000
4,2024/07/28,ops-cs-L1-en-chat,0.000000
...,...,...,...
8350,2026/02/04,ops-cs-L1-fr-chat,1.623029
8351,2026/02/04,ops-cs-L1-fr-email,2.901506
8352,2026/02/04,ops-cs-L1-it-call,0.146452
8353,2026/02/04,ops-cs-L1-it-chat,1.150446


---
###### Seasonally adjust the forecast with intra-week and intra-month seasonality:

In [11]:
parameters_month = dict(
    
    # month seasonality:
    sheet_id                    = '1LldywXEzrbLP8OiQcR2-OhEPo8qR3ZeBCACqf_aRuXQ',
    tab                         = 'intra-month_seasonality!A1:D'
)

In [12]:
parameters_week = dict(
    
    # week seasonality:
    sheet_id                    = '1LldywXEzrbLP8OiQcR2-OhEPo8qR3ZeBCACqf_aRuXQ',
    tab                         = 'intra-week_seasonality!A1:E'
)

In [13]:
# adjust seasonality function
def adj_seasonality(df,parameters_month=parameters_month,parameters_week=parameters_week):
    
    # reset index
    df = df.reset_index()
    
    # getting day of week and day of month
    df['dow'] = df['forecast_date'].dt.weekday+1
    df['dom']  = df['forecast_date'].dt.day
    
    # getting seasonality numbers
    month = fc.import_gsheet_to_df(parameters_month['sheet_id'], parameters_month['tab'])
    week = fc.import_gsheet_to_df(parameters_week['sheet_id'], parameters_week['tab'])
    
    # converting data types
    month['dom'] = month['dom'].astype('int32')
    month['intra_month_seasonality'] = month['intra_month_seasonality'].astype('float')
    week['dow'] = week['dow'].astype('int32')
    week['seasonality'] = week['seasonality'].astype('float')
    
    #merging with month
    df = df.merge(month,how='left',on=['business_line_alias','dom'])
    df = df.drop(columns=['avg_vol'])
    
    # merging with week
    df = df.merge(week,how='left',on=['business_line_alias','dow'])
    
    # adjusting the forecast
    df['adj_forecast'] = df['forecast']*(1+df['intra_month_seasonality'])*(1+df['seasonality'])
    
    # choosing columns
    df = df[['forecast_date','business_line_alias','adj_forecast']]
    
    # renaming column
    df = df.rename(columns={'adj_forecast':'forecast'})
    
    return df

In [14]:
# run the function
df_fc_grouped = adj_seasonality(df = df_fc_grouped)

---
###### Output the final forecast to the G-Sheet or CSV:

In [15]:

#df_fc_grouped['cohort_start_date'] = df_fc_grouped['cohort_start_date'].dt.strftime('%Y/%m/%d')
df_fc_grouped['forecast_date'] = df_fc_grouped['forecast_date'].dt.strftime('%Y/%m/%d')

In [16]:
#todo: output
#logging.warning('Missing volume distribution for following business line(s): ' + missing_groups_concat + '\n🚨 Cohorts from these business lines will be missing from the output!!')

#df_fc = df_fc.reset_index()
#df_fc['forecast_date'] = df_fc['forecast_date'].dt.strftime('%Y/%m/%d')

gsheet_export_params = dict(
    
    df                 = df_fc_grouped,
    gsheet_id          = '1a_v0g2y5DvqLDbXXJC0m7i3PjF2AseC3lqoRwlGxg1o',
    gsheet_tab_name    = 'kycc_cs_vol',
    include_df_headers = True,
    tab_colour         = (0.0, 0.0, 0.0) #RGB tab colour
)

fc.export_df_to_google_sheet(**gsheet_export_params)